# Amplitude altitudinal

## Importando pacotes e inicializando geemap

In [1]:
import os
import ee
import geemap

In [2]:
geemap.ee_initialize()


## Cálculo da amplitude altitudinal

A amplitude altitudinal foi calculada como a diferença entre a altitude máxima e mínima dentro da vizinhaça da célula focal. 
Como essa métrica é correlacionada com a variedade de formas de relevo, nós calculamos o resíduos de uma 
regressão (*Ordinary Least Squares*) entre as duas váriaveis. Assim, a amplitude altitudinal residual é independente da variedade de formas de relevo. Isso permite a identificação de locais que tenham maior variabilidade microclimática que aquela proporcionada pela variedade de formas de relevo, quando calcularmos o índice de diversidade da paisagem.

## Base de dados

Utilizamos o Modelo Digital de Elevação (DEM) do Merit-DEM [@yamazaki_high-accuracy_2017], na escala de 90 metros. O Merit-DEM é um produto global que combina dados dos satélites do Shuttle Radar Topography Mission (SRTM) [@farr_shuttle_2007] e Advanced Land Observing Satellite (ALOS) [@tadono_t_ishida_h_oda_f_naito_s_minakawa_k_iwamoto_h_last_precise_2014], permitindo a replicabilidade da metodologia em outras regiões. Além disso, o Merit-DEM corrige viéses de Modelo Digitais de Elevação gerados por imagens de satétite como *speckle noise*, *stripe noise*, *absolute bias* e *tree height bias* [@yamazaki_high-accuracy_2017]. A correção de *tree height bias* é especialmente importante para a Floresta Amazônica devido à sua densidade de árvores altas. 

A variedade de formas de relevo foi calculada anteriormente (veja o capítulo [Variedade de formas de relevo](https://resiliencia-climatica-brasil.github.io/diversity-resilience-python/1_landforms.html)) e está disponível como um *asset* em **[https://code.earthengine.google.com/?asset=projects/ee-lucasljardim9/assets/landform_variety](https://code.earthengine.google.com/?asset=projects/ee-lucasljardim9/assets/landform_variety)**.   

As análises foram rodadas no *Google Earth Engine* [@gorelick_google_2017], devido à demanda computacional do projeto, usando o pacote **geemap** [@wu_geemap_2020] em *Python* [@python_software_foundation_python_2023] como interface pela facilidade na documentação e reprodutividade das análises. O JupyterNotebook para a reprodução das análises pode ser encontrado em [https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/2_elevation_range.ipynb](https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/2_elevation_range.ipynb). 


## Códigos para o cálculo da amplitude altitudinal residual

Primeiro, importamos um vetor dos biomas do Brasil para extrairmos as coordenadas mínimas e máximas do Brasil.


In [3]:
# Importando mapa de biomas do IBGE para extrair as coordenadas mínimas e máximas do Brasil
regiao = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
regiao_box = regiao.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

Importamos os *rasters* do modelo digital de elevação e da variedade de formas de relevo e definimos a resolução da análise como aquela do modelo digital de elevação.

In [4]:
# Importando o modelo digital de elevação
DEM = ee.Image("MERIT/DEM/v1_0_3")

# Importando a variedade de landforms calculada anteriormente
landform_variety = ee.Image("projects/ee-lucasljardim9/assets/landform_variety")

#Escala dos rasters ~92 metros
escala = DEM.projection().nominalScale()

Extraímos do *raster* as células dentro da vizinhança (kernel circular com 5 células de raio, ~450 metros) da célula focal e salvamos como bandas de uma imagem. Assim, cada banda é um *stack* das células da vizinhança da célula focal, a primeira banda possui todas as primeiras células de cada célula focal, a segunda banda todas as segunda células, e assim por diante. 

In [5]:
# Tamanho do raio do kernel para o calculo da amplitude altitudinal
radius_pixels = 5

# Criando rasters da vizinhança de cada célula como bandas da imagem
neighbor = DEM.neighborhoodToBands(ee.Kernel.circle(ee.Number(radius_pixels)))

O *raster* *neighbor* criado acima possui as células da vizinhança como bandas. Assim, as primeiras células de cada banda são as células da vizinhança da primeira célula focal, organizadas como colunas (bandas). Ao calcularmos os valores máximos e mínimos para cada coluna de *neighbor*, estamos calculando os valores máximos e mínimos da vizinhança de cada célula focal. 

In [6]:
# Calcule o máximo da vizinhança
elevation_max = neighbor.reduce(ee.Reducer.max())

# Calcule o mínimo da vizinhança
elevation_min = neighbor.reduce(ee.Reducer.min())

Subtraindo os valores máximos e mínimos de cada célula focal e calculando o seu valor absoluto, temos a amplitude altitudinal para cada célula focal. Nós salvamos a amplitude altitudinal e variedade de formas de relevo como um *raster* com duas bandas, sendo a primeira banda a variável preditora da regressão e a segunda banda a variável resposta.

In [7]:
# Calcule a amplitude da vizinhança
elevation_range = elevation_max.subtract(elevation_min).abs()

# Crie uma imagem com as bandas de varidedade de formas de relevo e amplitude altitudinal 
# A primeira imagem é o x da regressão e a segunda é o y

elevation = (ee.Image.cat(landform_variety, elevation_range)
            .rename(['landform_variety', 'elevation_range']))

Dessa forma, aplicamos a regressão entre as variáveis.

In [9]:
# Rode uma regressão linear (OLS) entre varidade formas de relevo e amplitude altitudinal
regression = elevation.reduceRegion(**{
     'reducer': ee.Reducer.linearFit(),
     'geometry': regiao_box,
     'maxPixels': 1e13,
     'scale': escala
    })

Após a regressão, multiplicamos a variável preditora pelo coeficiente de regressão (*slope*) e adicionamos o valor do intercepto para predizermos os valores de amplitude altitudinal esperados pela regressão. Em seguida, subtraímos os valores de amplitude altitudinal pelos valores preditos pela regressão para calcularmos os resíduos do modelo.

In [10]:
# Calcule o valor predito, pela regressão, de amplitude altitudinal, sem intercepto 
pred = elevation.select('landform_variety').multiply(ee.Number(regression.get('scale')))

# Adicione o intercepto na predição
predict = pred.add(ee.Number(regression.get('offset')))

# Calcule o residuo da regressão
residuals = elevation.select('elevation_range').subtract(predict).rename(['residuals'])

Por fim, exportamos o raster de amplitude altitudinal residual como um *asset* do *Google Earth Engine*.

In [12]:
# Exporte a amplitude altitudinal residual como asset

assetId = "projects/ee-lucasljardim9/assets/elevation_range_residual"

geemap.ee_export_image_to_asset(
    residuals, 
    description='elevation_range_residual', 
    assetId=assetId, 
    region=regiao_box, 
    scale=escala, 
    maxPixels=1e13
)